In [1]:
from pyspark.sql import SparkSession
from soda.scan import Scan

## Carregando DataFrames

In [2]:
spark = SparkSession.builder.getOrCreate()

In [123]:
yellow_df = spark.read.option('inferSchema',True).parquet('data/raw/yellow/*/*')
yellow_df.createOrReplaceTempView("yellow_df")

fhv_df = spark.read.option('inferSchema',True).parquet('data/raw/fhv/*/*')
fhv_df.createOrReplaceTempView("fhv_df")

green_df = spark.read.option('inferSchema',True).parquet('data/raw/green/*/*')
green_df.createOrReplaceTempView("green_df")

## Análise dos DataSets

In [60]:
yellow_df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [61]:
spark.sql("SELECT MIN(trip_distance) as min, MAX(trip_distance) as max, PERCENTILE (trip_distance, 0.25) p_025, PERCENTILE (trip_distance, 0.5) p_05, PERCENTILE (trip_distance, 0.75) p_075, PERCENTILE (trip_distance, 0.9) as p_9, PERCENTILE (trip_distance, 0.99) as p_99  FROM yellow_df").show()

+---+---------+-----+----+-----+---+-----+
|min|      max|p_025|p_05|p_075|p_9| p_99|
+---+---------+-----+----+-----+---+-----+
|0.0|258928.15| 1.07| 1.8| 3.48|9.0|20.21|
+---+---------+-----+----+-----+---+-----+



Observamos que os valores de trip_distance (distancia de viagem medida em milhas) possuí valores outliers/incorretos em sua distribuição (0 e 258928.15). Através da análise de percentis, verificamos que até 99% do Dataset a distância da viagem chega até 20.21 milhas. Portanto, definimos que os valores mínimos de uma viagem deve ser maior que 0 e menor (ou igual) a 30

In [82]:
x = spark.sql("SELECT DISTINCT Affiliated_base_number FROM fhv_df").show()

+----------------------+
|Affiliated_base_number|
+----------------------+
|       B00625         |
|                B02629|
|                B01081|
|                B03148|
|                B02803|
|                B01616|
|                B00411|
|                B02932|
|                B00789|
|                B01553|
|                B02303|
|                B02876|
|                B02942|
|                BO3404|
|                B01341|
|                B03445|
|                B02535|
|                B03323|
|                B00856|
|                B02788|
+----------------------+
only showing top 20 rows



In [62]:
fhv_df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp_ntz (nullable = true)
 |-- dropOff_datetime: timestamp_ntz (nullable = true)
 |-- PUlocationID: double (nullable = true)
 |-- DOlocationID: double (nullable = true)
 |-- SR_Flag: integer (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [64]:
green_df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- lpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- lpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: integer (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- trip_type: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



## Execução de validação de dados com SODA

In [140]:
checks  ="""
checks for yellow_df:
  - row_count > 0
  - max(trip_distance) <= 30
  - min(trip_distance) > 0
  - min(passenger_count) > 0
  - max(passenger_count) < 5
  - no_datetime_less_than_2018 = 0:
      no_datetime_less_than_2018 query: SELECT COUNT(*) FROM yellow_df WHERE tpep_pickup_datetime < '01-01-2018' or tpep_dropoff_datetime < '01-01-2018'
  - pickup_lower_than_dropoff = 0:
      pickup_lower_than_dropoff query: SELECT COUNT(*) FROM yellow_df WHERE tpep_pickup_datetime > tpep_dropoff_datetime
  - duplicate_lines_lower_than = 0:
      duplicate_lines_lower_than query: SELECT COUNT(foo.*) FROM (SELECT VendorID ,tpep_pickup_datetime ,tpep_dropoff_datetime ,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag ,PULocationID ,DOLocationID ,payment_type ,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee, COUNT(*) FROM yellow_df GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19 HAVING COUNT(*) > 1) as foo
  - schema:
      name: Confirm that required columns are present
      fail:
        when required column missing: [VendorID ,tpep_pickup_datetime ,tpep_dropoff_datetime ,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag ,PULocationID ,DOLocationID ,payment_type ,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee]
        when wrong column type:
          VendorID: bigint
          tpep_pickup_datetime: timestamp_ntz
          tpep_dropoff_datetime: timestamp_ntz
          passenger_count: double
          trip_distance: double
          RatecodeID: double
          store_and_fwd_flag: string
          PULocationID: bigint
          DOLocationID: bigint
          payment_type: bigint
          fare_amount: double
          extra: double
          mta_tax: double
          tip_amount: double
          tolls_amount: double
          improvement_surcharge: double
          total_amount: double
          congestion_surcharge: double
          airport_fee: double


checks for fhv_df:
  - row_count > 0
  - invalid_percent(dispatching_base_num) = 0%:
      valid regex: ([B][0-9]{5})
  - invalid_percent(Affiliated_base_number) = 0%:
      valid regex: ([B][0-9]{5})
  - no_datetime_less_than_2018 = 0:
      no_datetime_less_than_2018 query: SELECT COUNT(*) FROM fhv_df WHERE pickup_datetime < '01-01-2018' or dropOff_datetime < '01-01-2018'
  - pickup_lower_than_dropoff = 0:
      pickup_lower_than_dropoff query: SELECT COUNT(*) FROM fhv_df WHERE pickup_datetime > dropOff_datetime
  - duplicate_lines_lower_than = 0:
      duplicate_lines_lower_than query: SELECT COUNT(foo.*) FROM (SELECT dispatching_base_num, pickup_datetime, dropOff_datetime, PUlocationID, DOlocationID, SR_Flag, Affiliated_base_number, COUNT(*) FROM fhv_df GROUP BY 1,2,3,4,5,6,7 HAVING COUNT(*) > 1) as foo
  - schema:
      name: Confirm that required columns are present
      fail:
        when required column missing: [dispatching_base_num, pickup_datetime, dropOff_datetime, PUlocationID, DOlocationID, SR_Flag, Affiliated_base_number]
        when wrong column type:
            dispatching_base_num: string
            pickup_datetime: timestamp_ntz
            dropOff_datetime: timestamp_ntz
            PUlocationID: double
            DOlocationID: double
            SR_Flag: int
            Affiliated_base_number: string


checks for green_df:
  - row_count > 0
  - min(passenger_count) > 0
  - max(passenger_count) < 5
  - no_datetime_less_than_2018 = 0:
      no_datetime_less_than_2018 query: SELECT COUNT(*) FROM green_df WHERE lpep_pickup_datetime < '01-01-2018' or lpep_dropoff_datetime < '01-01-2018'
  - pickup_lower_than_dropoff = 0:
      pickup_lower_than_dropoff query: SELECT COUNT(*) FROM green_df WHERE lpep_pickup_datetime > lpep_dropoff_datetime 
  - duplicate_lines_lower_than = 0:
      duplicate_lines_lower_than query: SELECT COUNT(foo.*) FROM (SELECT VendorID, lpep_pickup_datetime, lpep_dropoff_datetime, store_and_fwd_flag, RatecodeID, PULocationID, DOLocationID, passenger_count, trip_distance, fare_amount, extra, mta_tax, tip_amount, tolls_amount, ehail_fee, improvement_surcharge, total_amount, payment_type, trip_type, congestion_surcharge, COUNT(*) FROM green_df GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20 HAVING COUNT(*) > 1) as foo
  - schema:
      name: Confirm that required columns are present
      fail:
        when required column missing: [VendorID, lpep_pickup_datetime, lpep_dropoff_datetime, store_and_fwd_flag, RatecodeID, PULocationID, DOLocationID, passenger_count, trip_distance, fare_amount, extra, mta_tax, tip_amount, tolls_amount, ehail_fee, improvement_surcharge, total_amount, payment_type, trip_type, congestion_surcharge]
        when wrong column type:
            VendorID: bigint
            lpep_pickup_datetime: timestamp_ntz
            lpep_dropoff_datetime: timestamp_ntz
            store_and_fwd_flag: string
            RatecodeID: double
            PULocationID: bigint
            DOLocationID: bigint
            passenger_count: double
            trip_distance: double
            fare_amount: double
            extra: double
            mta_tax: double
            tip_amount: double
            tolls_amount: double
            ehail_fee: int
            improvement_surcharge: double
            total_amount: double
            payment_type: double
            trip_type: double
            congestion_surcharge: double
"""

In [141]:
scan = Scan()

scan.set_scan_definition_name("Datasets validation")
scan.set_data_source_name("spark_df")
scan.add_spark_session(spark)

scan.add_sodacl_yaml_str(checks)

scan.execute()
print(scan.get_logs_text())

INFO   | Soda Core 3.0.34
INFO   | Using DefaultSampler
INFO   | Scan summary:
INFO   | 15/23 checks PASSED: 
INFO   |     yellow_df in spark_df
INFO   |       row_count > 0 [PASSED]
INFO   |       no_datetime_less_than_2018 = 0 [PASSED]
INFO   |       duplicate_lines_lower_than = 0 [PASSED]
INFO   |       Confirm that required columns are present [PASSED]
INFO   |     fhv_df in spark_df
INFO   |       row_count > 0 [PASSED]
INFO   |       no_datetime_less_than_2018 = 0 [PASSED]
INFO   |       pickup_lower_than_dropoff = 0 [PASSED]
INFO   |       duplicate_lines_lower_than = 0 [PASSED]
INFO   |       Confirm that required columns are present [PASSED]
INFO   |       invalid_percent(dispatching_base_num) = 0% [PASSED]
INFO   |     green_df in spark_df
INFO   |       row_count > 0 [PASSED]
INFO   |       no_datetime_less_than_2018 = 0 [PASSED]
INFO   |       pickup_lower_than_dropoff = 0 [PASSED]
INFO   |       duplicate_lines_lower_than = 0 [PASSED]
INFO   |       Confirm that required c